# Notebook Contents
01. Importing Libraries
02. Importing Data
03. Wrangle df_customers
04. Consistency Checks
05. Merging Data
06. Exporting Data

# 01. Import Libraries

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Import Data

In [2]:
#Creating path
path =r'C:\Users\smflo\Documents\03-2023 Instacart Basket Analysis'

In [3]:
df_ords_prod_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_orders_products_merged_updated_2.pkl'))

In [4]:
df_customers = pd.read_csv(os.path.join(path, '02 Data','Original Data', 'customers.csv'),index_col = False)

# 03. Wrangle df_customers

In [5]:
#Checking output 
df_customers.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [6]:
#Dropping unnecessary columns
df_customers = df_customers.drop(columns = ['First Name','Surnam'])

In [7]:
#Checking output 
df_customers.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
#Renaming columns to make all lower case and easier to understand
df_customers.rename(columns = {'Gender' : 'gender', 'STATE' : 'state','Age' : 'age',
                               'n_dependants' : 'number_dependants', 'fam_status' : 'marital_status'}, inplace = True)

In [9]:
#Checking output 
df_customers.head()

,user_id,gender,state,age,date_joined,number_dependants,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [10]:
#Checking for missing data 
df_customers.isnull().sum()

user_id              0
gender               0
state                0
age                  0
date_joined          0
number_dependants    0
marital_status       0
income               0
dtype: int64

In [11]:
#Changing data type of 'user_id' to string
df_customers['user_id'] = df_customers['user_id'].astype('str')

In [12]:
#Changing data type of 'user_id' to string
df_ords_prod_merged['user_id'] = df_ords_prod_merged['user_id'].astype('str')

# 04. Consistency Checks

In [13]:
df_customers.describe()

,age,number_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [14]:
#Checking for duplicates
df_customers[df_customers.duplicated()]

,user_id,gender,state,age,date_joined,number_dependants,marital_status,income


No duplicates

In [15]:
#Checking for mixed-type data 
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

No mixed-type data

# 05. Merging Data

In [16]:
#Merging datasets 
df_ords_prod_cust = pd.merge(df_ords_prod_merged, df_customers, on = 'user_id', how = 'outer')

In [17]:
#Checking output
df_ords_prod_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_prior_orders,order_frequency_flag,gender,state,age,date_joined,number_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [18]:
#Checking shape
df_ords_prod_cust.shape

(32404859, 30)

In [19]:
#Checking value counts
df_ords_prod_cust['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

# 06. Exporting Data

In [20]:
#Export data to pkl
df_ords_prod_cust.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))